In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
import time
import pandas as pd
import pickle as pkl
import sqlite3

In [2]:
conn = sqlite3.connect("netflix_titles.db")

In [3]:
storys = pd.read_sql("select * from netflix_titles", conn)
storys.head()

,index,title,actors,release_year,rating,description,duration,ranking
0,0,Dick Johnson Is Dead,empty,2020,PG-13,"As her father nears the end of his life, filmm...",90 min,0.0
1,1,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021,TV-MA,"After crossing paths at a party, a Cape Town t...",2 Seasons,0.0
2,2,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021,TV-MA,To protect his family from a powerful drug lor...,1 Season,0.0
3,3,Jailbirds New Orleans,empty,2021,TV-MA,"Feuds, flirtations and toilet talk go down amo...",1 Season,0.0
4,4,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",2021,TV-MA,In a city of coaching centers known to train I...,2 Seasons,0.0


In [4]:
corpus_embeddings = torch.load('corpus_embeddings.pt', map_location=torch.device('cpu'))

In [5]:
model = SentenceTransformer('sentence-transformers/LaBSE')

In [6]:
# Load precomputed corpus embeddings
corpus_embeddings = torch.load('corpus_embeddings.pt', map_location=torch.device('cpu'))

# Connect to the database
conn = sqlite3.connect("netflix_titles.db")

# Read data from the database
storys = pd.read_sql("SELECT * FROM netflix_titles", conn)


In [7]:
def search(inp_question):
    start_time = time.time()
    question_embedding = model.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=75, score_function=util.dot_score)
    end_time = time.time()
    mean =  sum(list([float(x['score']) for x in hits[0]]))/len(hits[0])
    idx =  list([x['corpus_id'] for x in hits[0] if x['score']>mean])
    
    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    return storys.iloc[idx].sort_values(by='ranking', ascending=False)
    # return hits[0]

In [8]:
search('What the F* Is Going On?')

Input question: What the F* Is Going On?
Results (after 0.493 seconds):


,index,title,actors,release_year,rating,description,duration,ranking
3234,3234,What the F* Is Going On?,empty,2019,TV-MA,"Featuring extensive interviews, this documenta...",87 min,0.0
4614,4614,What the Jatt!!,"Harish Verma, Isha Rikhi, Vipul Roy, Binnu Dhi...",2015,TV-PG,A peasant from Punjab and his best friend pose...,140 min,0.0
1553,1553,The Netflix Afterparty: The Best Shows of The ...,"David Spade, London Hughes, Fortune Feimster",2020,TV-MA,"David Spade, Fortune Feimster and London Hughe...",59 min,0.0
3640,3640,My First First Love,"Ji Soo, Jung Chae-yeon, Jinyoung, Choi Ri, Kan...",2019,TV-14,"Due to various personal reasons, a group of Yu...",2 Seasons,0.0
3059,3059,Garfield's Pet Force,"Frank Welker, Vanessa Marshall, Gregg Berger, ...",2009,G,When an alien menace and her army of undead mi...,78 min,0.0
1437,1437,Surviving Death,empty,2021,TV-MA,What happens after we die? This docuseries exp...,1 Season,0.0
8419,8419,The Memphis Belle: A Story of a\nFlying Fortress,empty,1944,TV-PG,This documentary centers on the crew of the B-...,40 min,0.0
3141,3141,Karthik Calling Karthik,"Farhan Akhtar, Deepika Padukone, Ram Kapoor, V...",2010,TV-14,"Unlucky in love and bullied at work, an office...",135 min,0.0
7457,7457,Mini Wolf,empty,2014,TV-G,Mini-Wolf and his fun-loving pals friends are ...,2 Seasons,0.0
509,509,Ask the StoryBots,"Judy Greer, Erin Fitzgerald, Fred Tatasciore, ...",2019,TV-Y,Five curious little creatures track down the a...,3 Seasons,0.0


In [9]:
storys['release_year'].value_counts()

2018    1147
2017    1032
2019    1030
2020     953
2016     902
        ... 
1959       1
1925       1
1961       1
1947       1
1966       1
Name: release_year, Length: 74, dtype: int64

To evaluate the system using appropriate metrics like precision, recall, and F1-score, We can use the evaluate_user_feedback functio


In [16]:
import torch
import sqlite3

def evaluate_relevance(relevant_indices, system_recommendations):
    true_positives = len(set(relevant_indices) & set(system_recommendations))
    false_positives = len(set(system_recommendations) - set(relevant_indices))
    false_negatives = len(set(relevant_indices) - set(system_recommendations))
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    
    return precision, recall, f1_score

# Test the system with a sample query and known relevant indices
sample_query = 'What the F* Is Going On?'
known_relevant_indices = [462, 3234, 3527, 8133]  # Provide a list of known relevant indices 
# These indices likely represent documents in a dataset that are considered known and confirmed to be relevant for a particular task or evaluation.
search("secrets and lies ")


Input question: secrets and lies 
Results (after 0.253 seconds):


,index,title,actors,release_year,rating,description,duration,ranking
8604,8604,Top 10 Secrets and Mysteries,empty,2018,TV-14,This series investigates mysteries that persis...,1 Season,0.0
1877,1877,Private Lives,"Seohyun, Ko Kyung-pyo, Kim Hyo-jin, Kim Young-...",2020,TV-14,"In a world where data is no longer private, co...",1 Season,0.0
569,569,Dreamy Eyes,"Tran Nghia, Truc Anh, Tran Phong, Khanh Van, N...",2019,TV-14,"Through heartbreak and betrayal, Ngan's unrequ...",117 min,0.0
7813,7813,Queens vs. Kings,"Varun Thakur, Anu Menon, Rahul Subramanian, Si...",2018,TV-14,"No ""ism"" is left unturned in this celebrity ga...",1 Season,0.0
1170,1170,Secret Magic Control Agency,"Nicholas Corda, Sylvana Joyce, Alyson Leigh Ro...",2021,TV-Y7,Hansel and Gretel of fairy tale fame — now act...,105 min,0.0
5812,5812,Caught on Camera,Nick Wallis,2015,TV-MA,"Nick Wallis presents crimes caught on CCTV, ce...",3 Seasons,0.0
5806,5806,Witches: A Century of Murder,Suzannah Lipscomb,2015,TV-14,Historian Suzannah Lipscomb investigates the w...,1 Season,0.0
2604,2604,A Secret Love,empty,2020,TV-14,"Amid shifting times, two women kept their deca...",83 min,0.0
4478,4478,DreamWorks Spooky Stories,"Antonio Banderas, Hugh Laurie, Mike Myers, Ree...",2009,TV-PG,Shrek dares his friends to bring on the scary ...,1 Season,0.0
7978,7978,Secrets of Underground London,Samuel West,2014,TV-PG,Experts examine the ruins and relics hidden be...,55 min,0.0


In [12]:
from Interface import MyInterface

In [14]:
ui = MyInterface("Visal")
ui.say_hello()

Hello, Visal!
